[data source] https://github.com/lerocha/chinook-database

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('chinook.db')

In [3]:
cursor = conn.cursor()

In [4]:
# lets looks the tables present in the sql database
cursor.execute('SELECT type, name FROM sqlite_master WHERE type = "table"').fetchall()

[('table', 'album'),
 ('table', 'artist'),
 ('table', 'customer'),
 ('table', 'employee'),
 ('table', 'genre'),
 ('table', 'invoice'),
 ('table', 'invoice_line'),
 ('table', 'media_type'),
 ('table', 'playlist'),
 ('table', 'playlist_track'),
 ('table', 'track')]

In [5]:
pd.read_sql_query("SELECT type, name FROM sqlite_master WHERE type = 'table';", con=conn)

,type,name
0,table,album
1,table,artist
2,table,customer
3,table,employee
4,table,genre
5,table,invoice
6,table,invoice_line
7,table,media_type
8,table,playlist
9,table,playlist_track


#### As we can see there are total 11 tables in our data base.

[Q] For one single purchase (invoice_id) we want to know, for each track purchased:
```python
The id of the track.
The name of the track.
The name of media type of the track.
The price that the customer paid for the track.
The quantity of the track that was purchased.```

### Lets look at schema of database so that we can get an overall idea
![](chinook-schema.svg)

### To answer the above questions it looks like we need invoice_line, track and media_type tables
- Lets explore these tables first to get sense of the features

In [8]:
pd.read_sql_query("SELECT * FROM invoice_line LIMIT 5;", conn)

,invoice_line_id,invoice_id,track_id,unit_price,quantity
0,1,1,1158,0.99,1
1,2,1,1159,0.99,1
2,3,1,1160,0.99,1
3,4,1,1161,0.99,1
4,5,1,1162,0.99,1


In [9]:
pd.read_sql_query("SELECT * FROM track LIMIT 5;", conn)

,track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [10]:
pd.read_sql_query("SELECT * FROM media_type LIMIT 5;", conn)

,media_type_id,name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [26]:
# Since our question is for specific purchase or invoice, lets see for invoice id = 4
query = "SELECT t.track_id track_id, t.name track_name, m.name track_type,\
 t.unit_price, i.quantity FROM invoice_line i\
 JOIN track t ON i.track_id = t.track_id\
 JOIN media_type m ON t.media_type_id = m.media_type_id\
 WHERE invoice_id = 4"
pd.read_sql_query(query, conn)

,track_id,track_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",Protected AAC audio file,0.99,1
1,2560,Violent Pornography,MPEG audio file,0.99,1
2,3336,War Pigs,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,MPEG audio file,0.99,1
4,1872,Attitude,MPEG audio file,0.99,1
5,748,Dealer,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),MPEG audio file,0.99,1
7,2514,Spoonman,MPEG audio file,0.99,1


- customer with invoice id 4 has purchased total 8 tracks of each track price of 0.99
- there are 2 types of track types he purchased, one is protected AAC audio file and other is MPEG audio files
- Track id's and Track names can be seen from above pandas table.

## [Q] Include artist column to the above query
- From schema we can see artist column is not directly present in the track table.
- But instead the connection between track and artist table is album table.
- So lets join the track artist table using album table and see the artist name corresponding to each track.

In [28]:
# lets join artist name to our query
query = "SELECT t.track_id track_id, t.name track_name,\
 art.name artist_name, m.name track_type,\
 t.unit_price, i.quantity FROM invoice_line i\
 JOIN track t ON i.track_id = t.track_id\
 JOIN media_type m ON t.media_type_id = m.media_type_id\
 JOIN album alb ON t.album_id = alb.album_id\
 JOIN artist art ON alb.artist_id = art.artist_id\
 WHERE invoice_id = 4"
pd.read_sql_query(query, conn)

,track_id,track_name,artist_name,track_type,unit_price,quantity
0,3448,"Lamentations of Jeremiah, First Set \ Incipit ...",The King's Singers,Protected AAC audio file,0.99,1
1,2560,Violent Pornography,System Of A Down,MPEG audio file,0.99,1
2,3336,War Pigs,Cake,Purchased AAC audio file,0.99,1
3,829,Let's Get Rocked,Def Leppard,MPEG audio file,0.99,1
4,1872,Attitude,Metallica,MPEG audio file,0.99,1
5,748,Dealer,Deep Purple,MPEG audio file,0.99,1
6,1778,You're What's Happening (In The World Today),Marvin Gaye,MPEG audio file,0.99,1
7,2514,Spoonman,Soundgarden,MPEG audio file,0.99,1
